# Workspace for Project Euler problems

## Problem 49

In [1]:
import tools.primetools as pt
import numbers, math
import tools.misc as misc
import logging, sys, time
from functools import reduce

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

log = logging.getLogger("main")

# DEBUG, INFO, WARNING, ERROR, CRITICAL
log.setLevel(logging.INFO)

In [2]:
primes = pt.findSomePrimes(limit=10000)

In [4]:
ps = set(p for p in primes if p > 1000)

In [10]:
sorted("1324")

['1', '2', '3', '4']

In [31]:
def check(num, number_as_sorted_list):
    return num in ps and number_as_sorted_list == sorted(str(num))

def find_prime_anagrams():
    solutions = []
    for p in ps:
    #     print(p)   
        limit = p + (10000 - p)//2
        pl = sorted(str(p))
        for next_num in range(p + 18, limit, 18):
    #         print(next_num)
            if check(next_num, pl):
    #             print("found a candidate:", next_num)
                last_num = next_num*2 - p
                if check(last_num, pl):
#                     print("winner", p, next_num, last_num)
                    solutions.append((p, next_num, last_num))
    #     break
    return solutions
find_prime_anagrams()

[(2969, 6299, 9629), (1487, 4817, 8147)]

In [30]:
%timeit find_prime_anagrams()

50.3 ms ± 6.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
def get_seqs(num):
    seqs = []
    for i in range(18, 5000, 18):
        if num+i*2 > 9999:
            break
        seqs.append((num, num + i, num + i*2))
    return seqs

def get_grams_alec_way():
    solutions = []
    for p in ps:
        seqs = get_seqs(p)
        for seq in seqs:
            if all(p0 in ps for p0 in seq):
                if len(set(''.join(sorted(str(p0))) for p0 in seq)) == 1:
                    solutions.append(seq)
    return solutions
get_grams_alec_way()

[(2969, 6299, 9629), (1487, 4817, 8147)]

In [33]:
%timeit get_grams_alec_way()

250 ms ± 87.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
